<a href="https://colab.research.google.com/github/Psubinn/BigData-Analyst/blob/main/A_to_Z_%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_%EC%8B%A4%EC%8A%B51.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

fatal: destination path 'yemoonsaBigdata' already exists and is not an empty directory.


1. 실습 A to Z : Motor Trend Car Road Tests

(1) 사전 준비

In [2]:
import pandas as pd
import numpy as np

X_test = pd.read_csv("./yemoonsaBigdata/datasets/Part2/mpg_X_test.csv")
X_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/mpg_X_train.csv")
y_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/mpg_y_train.csv")

(2) 데이터 전처리

In [3]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          278 non-null    object 
 1   mpg           278 non-null    float64
 2   cylinders     278 non-null    int64  
 3   displacement  278 non-null    float64
 4   horsepower    274 non-null    float64
 5   weight        278 non-null    int64  
 6   acceleration  278 non-null    float64
 7   model_year    278 non-null    int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 17.5+ KB
None


In [4]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train[['horsepower']] = imputer.fit_transform(X_train[['horsepower']])
X_test[['horsepower']] = imputer.fit_transform(X_test[['horsepower']])

In [5]:
print(X_train.describe())

              mpg   cylinders  displacement  horsepower       weight  \
count  278.000000  278.000000    278.000000  278.000000   278.000000   
mean    23.732734    5.374101    189.994604  103.383212  2948.464029   
std      7.647295    1.677084    105.471423   38.695458   862.949746   
min     10.000000    3.000000     68.000000   46.000000  1613.000000   
25%     18.000000    4.000000     98.000000   75.000000  2206.250000   
50%     23.000000    4.000000    140.500000   90.500000  2737.500000   
75%     29.000000    6.000000    258.000000  118.750000  3560.000000   
max     46.600000    8.000000    455.000000  230.000000  5140.000000   

       acceleration  model_year  
count    278.000000  278.000000  
mean      15.580216   76.057554  
std        2.745907    3.605591  
min        8.000000   70.000000  
25%       14.000000   73.000000  
50%       15.500000   76.000000  
75%       17.000000   79.000000  
max       24.800000   82.000000  


In [6]:
COL_DEL = ['name']
COL_NUM = ['mpg','cylinders','displacement','horsepower','weight','acceleration','model_year']
COL_CAT = []
COL_Y = ['isUSA']

X_train = X_train.iloc[ : , 1 : ]
X_test = X_test.iloc[ : , 1 : ]

(3) 데이터 모형 구축

In [7]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr ,y_val = train_test_split(X_train, y_train, test_size=0.3)

In [8]:
# 스케일링

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[COL_NUM])
X_tr[COL_NUM] = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])

In [9]:
from sklearn.neighbors import KNeighborsClassifier
modelKNN = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
modelKNN.fit(X_tr, y_tr.values.ravel())

from sklearn.tree import DecisionTreeClassifier
modelDT = DecisionTreeClassifier(max_depth=10)
modelDT.fit(X_tr,y_tr)

DecisionTreeClassifier(max_depth=10)

(4) 데이터 모형 평가

In [10]:
y_val_pred = modelKNN.predict(X_val)

y_val_pred_probaKNN = modelKNN.predict_proba(X_val)
y_val_pred_probaDT = modelDT.predict_proba(X_val)

In [11]:
from sklearn.metrics import roc_auc_score

scoreKNN = roc_auc_score(y_val,y_val_pred_probaKNN[:,1])
scoreDT = roc_auc_score(y_val, y_val_pred_probaDT[:,1])

print(scoreKNN, scoreDT)

0.8876201923076923 0.8665865384615384


In [12]:
# 직접 여러 하이퍼파라미터를 학습하는 모형

best_model = None
best_score = 0

for i in range(2,10):
  model = KNeighborsClassifier(n_neighbors=i, metric='euclidean')
  model.fit(X_tr, y_tr.values.ravel())
  y_val_pred_proba = model.predict_proba(X_val)
  score = roc_auc_score(y_val, y_val_pred_proba[:,1])
  print(i,'개의 이웃 확인 : ', score)
  if best_score <= score:
    best_model = model

2 개의 이웃 확인 :  0.82421875
3 개의 이웃 확인 :  0.8341346153846154
4 개의 이웃 확인 :  0.8783052884615384
5 개의 이웃 확인 :  0.8876201923076923
6 개의 이웃 확인 :  0.8870192307692308
7 개의 이웃 확인 :  0.8846153846153847
8 개의 이웃 확인 :  0.8858173076923076
9 개의 이웃 확인 :  0.8834134615384616


In [13]:
print(best_model.predict_proba(X_test))

[[0.77777778 0.22222222]
 [0.22222222 0.77777778]
 [0.         1.        ]
 [0.         1.        ]
 [0.77777778 0.22222222]
 [0.66666667 0.33333333]
 [0.66666667 0.33333333]
 [0.66666667 0.33333333]
 [0.66666667 0.33333333]
 [0.55555556 0.44444444]
 [0.         1.        ]
 [0.55555556 0.44444444]
 [0.         1.        ]
 [0.44444444 0.55555556]
 [0.66666667 0.33333333]
 [0.88888889 0.11111111]
 [0.         1.        ]
 [0.         1.        ]
 [0.77777778 0.22222222]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.55555556 0.44444444]
 [0.77777778 0.22222222]
 [0.         1.        ]
 [0.         1.        ]
 [0.44444444 0.55555556]
 [0.88888889 0.11111111]
 [0.77777778 0.22222222]
 [0.66666667 0.33333333]
 [0.22222222 0.77777778]
 [0.         1.        ]
 [0.44444444 0.55555556]
 [0.55555556 0.44444444]
 [0.         1.        ]
 [0.88888889 0.11111111]
 [0.33333333 0.66666667]


- 미국에 해당하는 경우가 1이므로, 두 번째 줄에 해당하는 것을 추출한다. ( [ : , 1 ] )

In [14]:
pred = best_model.predict_proba(X_test)[:,1]
print(pred)

[0.22222222 0.77777778 1.         1.         0.22222222 0.33333333
 0.33333333 0.33333333 0.33333333 0.44444444 1.         0.44444444
 1.         0.55555556 0.33333333 0.11111111 1.         1.
 0.22222222 1.         1.         1.         1.         1.
 1.         0.44444444 0.22222222 1.         1.         0.55555556
 0.11111111 0.22222222 0.33333333 0.77777778 1.         0.55555556
 0.44444444 1.         0.11111111 0.66666667 1.         0.11111111
 0.11111111 0.55555556 0.22222222 0.22222222 0.44444444 1.
 0.22222222 1.         0.         0.22222222 1.         0.22222222
 1.         1.         1.         0.88888889 1.         0.22222222
 0.88888889 0.77777778 1.         1.         0.77777778 1.
 1.         0.44444444 0.77777778 0.11111111 0.88888889 1.
 0.33333333 0.         1.         0.11111111 1.         0.33333333
 0.44444444 1.         0.88888889 1.         0.22222222 1.
 1.         0.88888889 0.22222222 0.11111111 0.33333333 1.
 1.         1.         1.         1.         1.    

In [15]:
pd.DataFrame({'isUSA' : pred}).to_csv('./yemoonsaBigdata/res/003000000.csv', index=False)